In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import datasets


# Tabela 1613 - Área destinada à colheita, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras permanentes [1]


## Tabela 1: Quantidade produzida (Toneladas) 
## Tabela 2: Valor da produção (Mil Reais)

###### OBS: Valor da produção: Variável derivada calculada pela média ponderada das informações de quantidade e preço médio corrente pago ao produtor, de acordo com os períodos de colheita e comercialização de cada produto. As despesas de frete, taxas e impostos não são incluídas no preço

Referências:
[1] https://sidra.ibge.gov.br/tabela/1613


# **Proposta** 

Prever qual será a produção dos municípios em 2018 baseado na produção dos anos anteriores. Utilizaremos os métodos 'KNN' e 'Random Forest' de regressão.

In [2]:
path_qnt_produzida  = "data/1613_qnt_produzida.csv"
path_valor_producao  = "data/1613_valor_producao.csv"

lista_municipios_PE = ['Sertânia', 'Floresta', 'Arcoverde e Pesqueira'] 
lista_municipios_PB = ['Monteiro', 'Campina Grande', 'Barra de Santana', 'Caturité', 'Queimadas', 'Pocinhos', 
                        'Lagoa Seca', 'Matinhas', 'São Sebastião de Lagoa de Roça', 'Alagoa Nova', 'Boqueirão',
                        'Boa Vista', 'Soledade', 'Juazeirinho', 'Cubati', 'Pedra Lavrada', 'Olivedos', 'Seridó',
                        'Cabaceiras', 'Congo', 'Coxixola', 'Sumé', 'Prata', 'Ouro Velho', 'Amparo', 'Serra Branca', 
                        'São José dos Cordeiros', 'São João do Cariri', 'Parari', 'Gurjão', 
                        'Livramento', 'Santa Luzia do Cariri']




df_qnt_produzida  = pd.read_csv(path_qnt_produzida, na_values=['NULL'], sep=";")
#df_qnt_produzida.head()
df_qnt_produzida.columns.tolist()


#df_qnt_produzida[df_qnt_produzida['Município'] == "tt0111161"]





['Nível',
 'Cód.',
 'Município',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018']

In [3]:
len(df_qnt_produzida)


5531

In [4]:
df_valor_producao = pd.read_csv(path_valor_producao, na_values=['NULL'], sep=";", skiprows=[1])
df_valor_producao

,Nível,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,MU,1100015.0,Alta Floresta D'Oeste (RO),11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142
1,MU,1100023.0,Ariquemes (RO),23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562
2,MU,1100031.0,Cabixi (RO),1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005
3,MU,1100049.0,Cacoal (RO),19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178
4,MU,1100056.0,Cerejeiras (RO),834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5525,MU,5222054.0,Vicentinópolis (GO),...,...,...,...,...,...,...,...,...,...,...,-,-,-
5526,MU,5222203.0,Vila Boa (GO),1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65,-
5527,MU,5222302.0,Vila Propício (GO),748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010,9377
5528,MU,5300108.0,Brasília (DF),20726,27870,28141,32164,34346,43376,47189,50730,46623,57819,70800,54820,98251,66182


Deletando a coluna Nível

In [5]:
del df_valor_producao['Nível']



In [6]:
df_valor_producao.dropna(inplace=True)
df_valor_producao['Município']

0       Alta Floresta D'Oeste (RO)
1                   Ariquemes (RO)
2                      Cabixi (RO)
3                      Cacoal (RO)
4                  Cerejeiras (RO)
                   ...            
5524               Vianópolis (GO)
5525           Vicentinópolis (GO)
5526                 Vila Boa (GO)
5527            Vila Propício (GO)
5528                 Brasília (DF)
Name: Município, Length: 5529, dtype: object

In [7]:
df_qnt_produzida = pd.read_csv(path_valor_producao, na_values=['NULL'], sep=";", skiprows=[1])
df_qnt_produzida.head()

,Nível,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,MU,1100015.0,Alta Floresta D'Oeste (RO),11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142
1,MU,1100023.0,Ariquemes (RO),23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562
2,MU,1100031.0,Cabixi (RO),1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005
3,MU,1100049.0,Cacoal (RO),19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178
4,MU,1100056.0,Cerejeiras (RO),834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028


In [8]:
del df_qnt_produzida['Nível']


In [9]:
df_qnt_produzida.dropna(inplace=True)
df_qnt_produzida['Município']

0       Alta Floresta D'Oeste (RO)
1                   Ariquemes (RO)
2                      Cabixi (RO)
3                      Cacoal (RO)
4                  Cerejeiras (RO)
                   ...            
5524               Vianópolis (GO)
5525           Vicentinópolis (GO)
5526                 Vila Boa (GO)
5527            Vila Propício (GO)
5528                 Brasília (DF)
Name: Município, Length: 5529, dtype: object

##### Recortando os strings a coluna municípios com os dados dos estados

In [10]:
df_valor_producao['Estado']=df_valor_producao['Município'].str.slice(-3,-1)
df_valor_producao['Município']=df_valor_producao['Município'].str.slice(stop=-4)
df_valor_producao = df_valor_producao.replace({'-' : 0, '...' : 0})
df_valor_producao

,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Estado
0,1100015.0,Alta Floresta D'Oeste,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142,RO
1,1100023.0,Ariquemes,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562,RO
2,1100031.0,Cabixi,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005,RO
3,1100049.0,Cacoal,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178,RO
4,1100056.0,Cerejeiras,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028,RO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5222005.0,Vianópolis,196,237,222,265,848,216,1890,973,131,111,75,98,218,204,GO
5525,5222054.0,Vicentinópolis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GO
5526,5222203.0,Vila Boa,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65,0,GO
5527,5222302.0,Vila Propício,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010,9377,GO


In [11]:
df_qnt_produzida['Estado']=df_qnt_produzida['Município'].str.slice(-3,-1)
df_qnt_produzida['Município']=df_qnt_produzida['Município'].str.slice(stop=-4)
df_qnt_produzida = df_qnt_produzida.replace({'-' : 0, '...' : 0})
df_qnt_produzida

,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Estado
0,1100015.0,Alta Floresta D'Oeste,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142,RO
1,1100023.0,Ariquemes,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562,RO
2,1100031.0,Cabixi,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005,RO
3,1100049.0,Cacoal,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178,RO
4,1100056.0,Cerejeiras,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028,RO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5222005.0,Vianópolis,196,237,222,265,848,216,1890,973,131,111,75,98,218,204,GO
5525,5222054.0,Vicentinópolis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GO
5526,5222203.0,Vila Boa,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65,0,GO
5527,5222302.0,Vila Propício,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010,9377,GO


#### Transformando as colunas dos anos de object para numérica

In [12]:
colunas = ['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
df_valor_producao[colunas]=df_valor_producao[colunas].apply(pd.to_numeric,errors='coerce', axis=1)
df_valor_producao['2009'].dtype

dtype('int64')

In [13]:
df_qnt_produzida[colunas]=df_qnt_produzida[colunas].apply(pd.to_numeric,errors='coerce', axis=1)
df_qnt_produzida['2015'].dtype

dtype('int64')

### Fazendo o filtro e coluna das regiões

Criando listas com cada estado em sua região

In [14]:
sudeste= ['SP','MG','ES','RJ']
nordeste=['MA','PI','CE','RN','PE','PB','SE','AL','BA']
centrooeste=['MT','MS','GO']
norte=['AM','RR','AP','PA','TO','RO','AC']
sul=['PR','RS','SC']

Criando a coluna região no dataframe

In [15]:
df_valor_producao['Regiao']=len(df_valor_producao['Estado'])

Alocando o nome de cada região em cada célula de acordo com as listas acima

In [16]:
for index, estado in enumerate(df_valor_producao['Estado']):
    #print(index, estado)
    if estado in sudeste:
        df_valor_producao.loc[index,'Regiao'] = 'Sudeste'
    elif estado in nordeste:
        df_valor_producao.loc[index,'Regiao'] = 'Nordeste'
    elif estado in centrooeste:
        df_valor_producao.loc[index,'Regiao'] = 'Centro Oeste'
    elif estado in norte:
        df_valor_producao.loc[index,'Regiao'] = 'Norte'
    else:
        df_valor_producao.loc[index,'Regiao'] = 'Sul'    

In [17]:
df_valor_producao

,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Estado,Regiao
0,1100015.0,Alta Floresta D'Oeste,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142,RO,Norte
1,1100023.0,Ariquemes,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562,RO,Norte
2,1100031.0,Cabixi,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005,RO,Norte
3,1100049.0,Cacoal,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178,RO,Norte
4,1100056.0,Cerejeiras,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028,RO,Norte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5222005.0,Vianópolis,196,237,222,265,848,216,1890,973,131,111,75,98,218,204,GO,Centro Oeste
5525,5222054.0,Vicentinópolis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GO,Centro Oeste
5526,5222203.0,Vila Boa,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65,0,GO,Centro Oeste
5527,5222302.0,Vila Propício,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010,9377,GO,Centro Oeste


#### Fazendo o mesmo filtro para o dataframe df_qnt_produzida

In [18]:
df_qnt_produzida['Regiao']=len(df_qnt_produzida['Estado'])

In [19]:
for index, estado in enumerate(df_qnt_produzida['Estado']):
    if estado in sudeste:
        df_qnt_produzida.loc[index,'Regiao'] = 'Sudeste'
    elif estado in nordeste:
        df_qnt_produzida.loc[index,'Regiao'] = 'Nordeste'
    elif estado in centrooeste:
        df_qnt_produzida.loc[index,'Regiao'] = 'Centro Oeste'
    elif estado in norte:
        df_qnt_produzida.loc[index,'Regiao'] = 'Norte'
    else:
        df_qnt_produzida.loc[index,'Regiao'] = 'Sul'    

In [20]:
df_qnt_produzida

,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Estado,Regiao
0,1100015.0,Alta Floresta D'Oeste,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142,RO,Norte
1,1100023.0,Ariquemes,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562,RO,Norte
2,1100031.0,Cabixi,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005,RO,Norte
3,1100049.0,Cacoal,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178,RO,Norte
4,1100056.0,Cerejeiras,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028,RO,Norte
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5222005.0,Vianópolis,196,237,222,265,848,216,1890,973,131,111,75,98,218,204,GO,Centro Oeste
5525,5222054.0,Vicentinópolis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GO,Centro Oeste
5526,5222203.0,Vila Boa,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65,0,GO,Centro Oeste
5527,5222302.0,Vila Propício,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010,9377,GO,Centro Oeste


Avaliando estatística básica de cada dataframe usando o .describe()

In [21]:
np.array(df_qnt_produzida['2005'].values, dtype=int)

array([11952, 23196,  1160, ...,  1086,   748, 20726])

In [22]:
df_valor_producao.describe()

,Cód.,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,5.529000e+03,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5.529000e+03,5529.000000,5.529000e+03,5529.000000
mean,3.253657e+06,3687.432266,4708.302044,4509.592693,5108.010309,4830.328812,5930.887683,7139.693254,6908.841924,6581.056430,7668.280159,8.721002e+03,10487.376379,1.017439e+04,10947.778622
std,9.845776e+05,11859.927851,16078.161827,15773.418836,16677.751276,15332.048135,19013.098303,21131.898868,21754.646996,21473.806932,23878.622994,3.712784e+04,33407.321699,3.479464e+04,36737.034026
min,1.100015e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,2.512036e+06,100.000000,109.000000,113.000000,123.000000,118.000000,127.000000,147.000000,117.000000,118.000000,125.000000,1.250000e+02,126.000000,1.060000e+02,120.000000
50%,3.146255e+06,442.000000,477.000000,505.000000,540.000000,520.000000,565.000000,683.000000,608.000000,645.000000,702.000000,7.370000e+02,828.000000,7.440000e+02,790.000000
75%,4.119202e+06,2168.000000,2534.000000,2516.000000,2839.000000,2766.000000,3004.000000,3788.000000,3384.000000,3749.000000,3974.000000,4.431000e+03,5368.000000,5.174000e+03,5282.000000
max,5.300108e+06,351799.000000,423457.000000,424957.000000,559038.000000,440247.000000,616868.000000,511735.000000,615623.000000,915114.000000,640855.000000,1.767972e+06,915565.000000,1.021864e+06,931317.000000


In [23]:
df_qnt_produzida.describe()

,Cód.,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
count,5.529000e+03,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5529.000000,5.529000e+03,5529.000000,5.529000e+03,5529.000000
mean,3.253657e+06,3687.432266,4708.302044,4509.592693,5108.010309,4830.328812,5930.887683,7139.693254,6908.841924,6581.056430,7668.280159,8.721002e+03,10487.376379,1.017439e+04,10947.778622
std,9.845776e+05,11859.927851,16078.161827,15773.418836,16677.751276,15332.048135,19013.098303,21131.898868,21754.646996,21473.806932,23878.622994,3.712784e+04,33407.321699,3.479464e+04,36737.034026
min,1.100015e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,2.512036e+06,100.000000,109.000000,113.000000,123.000000,118.000000,127.000000,147.000000,117.000000,118.000000,125.000000,1.250000e+02,126.000000,1.060000e+02,120.000000
50%,3.146255e+06,442.000000,477.000000,505.000000,540.000000,520.000000,565.000000,683.000000,608.000000,645.000000,702.000000,7.370000e+02,828.000000,7.440000e+02,790.000000
75%,4.119202e+06,2168.000000,2534.000000,2516.000000,2839.000000,2766.000000,3004.000000,3788.000000,3384.000000,3749.000000,3974.000000,4.431000e+03,5368.000000,5.174000e+03,5282.000000
max,5.300108e+06,351799.000000,423457.000000,424957.000000,559038.000000,440247.000000,616868.000000,511735.000000,615623.000000,915114.000000,640855.000000,1.767972e+06,915565.000000,1.021864e+06,931317.000000


## Numerando as classes

#### Primeiro df_valor_producao

In [24]:
from pandas.api.types import is_string_dtype

#ordenando as regiões
regiao_ordenada = df_valor_producao['Regiao'].astype('category').cat.as_ordered()
regiao_ordenada

0              Norte
1              Norte
2              Norte
3              Norte
4              Norte
            ...     
5524    Centro Oeste
5525    Centro Oeste
5526    Centro Oeste
5527    Centro Oeste
5528             Sul
Name: Regiao, Length: 5529, dtype: category
Categories (5, object): [Centro Oeste < Nordeste < Norte < Sudeste < Sul]

In [25]:
#renomeando as células de regiões para valores: centro oeste = 0, nordeste=1, norte = 2, sudeste = 3 e sul=4
regiao_ordenada.cat.categories
df_valor_producao['Regiao']=regiao_ordenada.cat.rename_categories(range(len(regiao_ordenada.cat.categories)))
df_valor_producao

,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Estado,Regiao
0,1100015.0,Alta Floresta D'Oeste,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142,RO,2
1,1100023.0,Ariquemes,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562,RO,2
2,1100031.0,Cabixi,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005,RO,2
3,1100049.0,Cacoal,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178,RO,2
4,1100056.0,Cerejeiras,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028,RO,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5222005.0,Vianópolis,196,237,222,265,848,216,1890,973,131,111,75,98,218,204,GO,0
5525,5222054.0,Vicentinópolis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GO,0
5526,5222203.0,Vila Boa,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65,0,GO,0
5527,5222302.0,Vila Propício,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010,9377,GO,0


In [26]:
# para os estados
estados_ordenado = df_valor_producao['Estado'].astype('category').cat.as_ordered()
estados_ordenado

0       RO
1       RO
2       RO
3       RO
4       RO
        ..
5524    GO
5525    GO
5526    GO
5527    GO
5528    DF
Name: Estado, Length: 5529, dtype: category
Categories (27, object): [AC < AL < AM < AP ... SC < SE < SP < TO]

In [27]:
estados_ordenado.cat.categories
df_valor_producao['Estado']=estados_ordenado.cat.rename_categories(range(len(estados_ordenado.cat.categories)))
df_valor_producao

,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Estado,Regiao
0,1100015.0,Alta Floresta D'Oeste,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142,20,2
1,1100023.0,Ariquemes,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562,20,2
2,1100031.0,Cabixi,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005,20,2
3,1100049.0,Cacoal,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178,20,2
4,1100056.0,Cerejeiras,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028,20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5222005.0,Vianópolis,196,237,222,265,848,216,1890,973,131,111,75,98,218,204,8,0
5525,5222054.0,Vicentinópolis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0
5526,5222203.0,Vila Boa,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65,0,8,0
5527,5222302.0,Vila Propício,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010,9377,8,0


#### Agora com df_qnt_produzida

In [28]:
#ordenando as regiões
regiao_ordenada = df_qnt_produzida['Regiao'].astype('category').cat.as_ordered()
regiao_ordenada

0              Norte
1              Norte
2              Norte
3              Norte
4              Norte
            ...     
5524    Centro Oeste
5525    Centro Oeste
5526    Centro Oeste
5527    Centro Oeste
5528             Sul
Name: Regiao, Length: 5529, dtype: category
Categories (5, object): [Centro Oeste < Nordeste < Norte < Sudeste < Sul]

In [29]:
#renomeando as células de regiões para valores: centro oeste = 0, nordeste=1, norte = 2, sudeste = 3 e sul=4
regiao_ordenada.cat.categories
df_qnt_produzida['Regiao']=regiao_ordenada.cat.rename_categories(range(len(regiao_ordenada.cat.categories)))
df_valor_producao

,Cód.,Município,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Estado,Regiao
0,1100015.0,Alta Floresta D'Oeste,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674,75142,20,2
1,1100023.0,Ariquemes,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230,12562,20,2
2,1100031.0,Cabixi,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899,3005,20,2
3,1100049.0,Cacoal,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288,108178,20,2
4,1100056.0,Cerejeiras,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053,1028,20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,5222005.0,Vianópolis,196,237,222,265,848,216,1890,973,131,111,75,98,218,204,8,0
5525,5222054.0,Vicentinópolis,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0
5526,5222203.0,Vila Boa,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65,0,8,0
5527,5222302.0,Vila Propício,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010,9377,8,0


# Aplicando regressores para prever o quanto foi produzido em 2018

Criando a matriz X

In [30]:
X = df_valor_producao[['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']]
X

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674
1,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230
2,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899
3,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288
4,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,196,237,222,265,848,216,1890,973,131,111,75,98,218
5525,0,0,0,0,0,0,0,0,0,0,0,0,0
5526,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65
5527,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010


Criando o vetor y

In [31]:
y=df_valor_producao['2018']
y

0        75142
1        12562
2         3005
3       108178
4         1028
         ...  
5524       204
5525         0
5526         0
5527      9377
5528     66182
Name: 2018, Length: 5529, dtype: int64

Separando meus dados entre os de treinamento e teste

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [33]:
# n_neighbors = 15
model = KNeighborsRegressor(n_neighbors=83, weights='distance')
model.fit(X_train, y_train)
predicted= model.predict(X_test)

In [34]:
from sklearn.metrics import r2_score

In [35]:
r2_score(y_test,predicted)

0.7997352320927292

In [36]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
predicted= m.predict(X_test)

In [37]:
r2_score(y_test,predicted)

0.9255562545484951

Fazendo para o dataframe de quantidade produzida

In [38]:
X = df_qnt_produzida[['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017']]
X

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,11952,11650,24266,35721,30532,28620,17663,29580,30500,37672,54258,70652,69674
1,23196,15803,20636,23523,23875,30022,29103,29165,22234,7985,12553,13553,13230
2,1160,446,1965,1808,1582,1754,1141,187,1981,3020,3085,3293,2899
3,19384,18469,30987,43242,31643,53013,37463,52983,53823,54165,51847,74426,156288
4,834,790,1288,1232,1073,1093,577,866,1013,1001,1019,1124,1053
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5524,196,237,222,265,848,216,1890,973,131,111,75,98,218
5525,0,0,0,0,0,0,0,0,0,0,0,0,0
5526,1086,1098,1105,1120,1318,1316,1347,1663,1618,1865,2032,2024,65
5527,748,1020,1200,1620,1350,1861,2800,2660,2694,9612,11566,10735,14010


In [39]:
y=df_qnt_produzida['2018']
y

0        75142
1        12562
2         3005
3       108178
4         1028
         ...  
5524       204
5525         0
5526         0
5527      9377
5528     66182
Name: 2018, Length: 5529, dtype: int64

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [41]:
# n_neighbors = 15
model = KNeighborsRegressor(n_neighbors=83, weights='distance')
model.fit(X_train, y_train)
predicted= model.predict(X_test)

In [42]:
r2_score(y_test,predicted)

0.7997352320927292

In [43]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
predicted= m.predict(X_test)

In [44]:
r2_score(y_test,predicted)

0.9136295840791481

# Tentando usar o classificador para prever a região

In [56]:
df_qnt_produzida_mean=df_qnt_produzida[['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']].mean(axis=1)
df_valor_producao_mean=df_valor_producao[['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']].mean(axis=1)

In [59]:
df_qnt_produzida_sum=df_qnt_produzida[['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']].sum(axis=1)
df_valor_producao_mean=df_valor_producao[['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']].mean(axis=1)
df_qnt_produzida_sum

0       527882
1       277440
2        27326
3       785911
4        13991
         ...  
5524      5684
5525         0
5526     17657
5527     71253
5528    679037
Length: 5529, dtype: int64